# Model Selection for SLP Prediction

This notebook performs model selection to predict the `slp` column using various machine learning algorithms with time series cross-validation.


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')


## 1. Load and Prepare Data


In [15]:
# Load the dataset
df = pd.read_csv('dataset/data_v2_full.csv', sep=';', decimal=',')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Dataset shape: (3560, 26)

Columns: ['date', 'entry', 'rlm', 'slp', 'day_of_year', 'day_of_week', 'holiday', 'weathercode', 'temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'apparent_temperature_max', 'apparent_temperature_min', 'apparent_temperature_mean', 'sunrise', 'sunset', 'daylight_duration', 'sunshine_duration', 'rain_sum', 'snowfall_sum', 'precipitation_hours', 'windspeed_10m_max', 'windgusts_10m_max', 'winddirection_10m_dominant', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration']


,date,entry,rlm,slp,day_of_year,day_of_week,holiday,weathercode,temperature_2m_max,temperature_2m_min,...,daylight_duration,sunshine_duration,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2016-01-01 00:00:00+00:00,4.501250e+06,2.565526e+06,1.935724e+06,1,4,1,3,4.3,0.1,...,27973.34,17706.46,0.0,0.0,0.0,9.0,18.4,208,2.64,0.21
1,2016-01-02 00:00:00+00:00,5.448037e+06,3.037879e+06,2.410158e+06,2,5,0,3,2.5,-5.3,...,28047.14,2501.06,0.0,0.0,0.0,24.0,48.2,97,1.34,0.5
2,2016-01-03 00:00:00+00:00,6.472305e+06,3.462336e+06,3.009970e+06,3,6,0,3,-5.4,-8.4,...,28127.21,21285.25,0.0,0.0,0.0,21.5,44.3,102,3.38,0.68
3,2016-01-04 00:00:00+00:00,7.766598e+06,4.565146e+06,3.201452e+06,4,0,0,3,-3.5,-7.9,...,28213.34,9701.3,0.0,0.0,0.0,18.9,39.2,97,2.47,0.61
4,2016-01-05 00:00:00+00:00,7.842385e+06,4.672447e+06,3.169938e+06,5,1,0,73,-5.2,-6.4,...,28305.35,0.0,0.0,2.45,14.0,16.2,33.5,84,1.0,0.46


In [16]:
# Remove 'rlm' and/or 'entry' columns if present
to_remove = [col for col in ['rlm', 'entry'] if col in df.columns]
if to_remove:
    df_clean = df.drop(columns=to_remove)
    print(f"Removed columns: {to_remove}")
else:
    df_clean = df.copy()
    print("Columns 'rlm' and 'entry' not found, dataset unchanged.")

print(f"Dataset shape after removing rlm and entry (if present): {df_clean.shape}")
print(f"\nRemaining columns: {df_clean.columns.tolist()}")


Removed columns: ['rlm', 'entry']
Dataset shape after removing rlm and entry (if present): (3560, 24)

Remaining columns: ['date', 'slp', 'day_of_year', 'day_of_week', 'holiday', 'weathercode', 'temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'apparent_temperature_max', 'apparent_temperature_min', 'apparent_temperature_mean', 'sunrise', 'sunset', 'daylight_duration', 'sunshine_duration', 'rain_sum', 'snowfall_sum', 'precipitation_hours', 'windspeed_10m_max', 'windgusts_10m_max', 'winddirection_10m_dominant', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration']


In [17]:
# Parse date and sort by date (important for time series)
df_clean['date'] = pd.to_datetime(df_clean['date'])
df_clean = df_clean.sort_values('date').reset_index(drop=True)


In [18]:
# Separate features and target
X = df_clean.drop(columns=['date', 'slp'])
y = df_clean['slp']

# Define feature types for proper preprocessing
boolean_cols = ['holiday']
categorical_cols = ['weathercode']  # Leave as-is for tree-based models
cyclical_cols = ['day_of_week', 'day_of_year', 'winddirection_10m_dominant']

# All other columns are continuous and should be scaled
continuous_cols = [col for col in X.columns 
                   if col not in boolean_cols + categorical_cols + cyclical_cols]

# Convert continuous columns to numeric (CSV has mixed decimal formats: some use ',' some use '.')
for col in continuous_cols:
    X[col] = pd.to_numeric(X[col], errors='coerce')

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nBoolean columns (not scaled): {boolean_cols}")
print(f"Categorical columns (not scaled): {categorical_cols}")
print(f"Cyclical columns (will be sin/cos encoded): {cyclical_cols}")
print(f"Continuous columns (will be scaled): {continuous_cols}")


Features shape: (3560, 22)
Target shape: (3560,)

Boolean columns (not scaled): ['holiday']
Categorical columns (not scaled): ['weathercode']
Cyclical columns (will be sin/cos encoded): ['day_of_week', 'day_of_year', 'winddirection_10m_dominant']
Continuous columns (will be scaled): ['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'apparent_temperature_max', 'apparent_temperature_min', 'apparent_temperature_mean', 'sunrise', 'sunset', 'daylight_duration', 'sunshine_duration', 'rain_sum', 'snowfall_sum', 'precipitation_hours', 'windspeed_10m_max', 'windgusts_10m_max', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration']


## 2. Feature Preprocessing

- **Boolean features** (holiday): Left unchanged (0/1)
- **Categorical features** (weathercode): Left unchanged (tree-based models handle them well)
- **Cyclical features** (day_of_week, day_of_year, winddirection): Sin/cos encoding to preserve circular nature
- **Continuous features**: StandardScaler normalization

In [19]:
# Apply cyclical encoding for cyclical features
# This preserves the circular nature of these variables (e.g., day 365 is close to day 1)

def cyclical_encode(df, col, max_val):
    """Encode a cyclical feature using sine and cosine transformation."""
    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    return df

# Create a copy and apply cyclical encoding
X_encoded = X.copy()

# Encode day_of_week (0-6, period=7)
X_encoded = cyclical_encode(X_encoded, 'day_of_week', 7)

# Encode day_of_year (1-366, period=366)
X_encoded = cyclical_encode(X_encoded, 'day_of_year', 366)

# Encode wind direction (0-360 degrees, period=360)
X_encoded = cyclical_encode(X_encoded, 'winddirection_10m_dominant', 360)

# Drop original cyclical columns (replaced by sin/cos versions)
X_encoded = X_encoded.drop(columns=cyclical_cols)

print(f"Shape after cyclical encoding: {X_encoded.shape}")
print(f"\nNew cyclical features added:")
for col in cyclical_cols:
    print(f"  {col} -> {col}_sin, {col}_cos")
print(f"\nAll features: {X_encoded.columns.tolist()}")


Shape after cyclical encoding: (3560, 25)

New cyclical features added:
  day_of_week -> day_of_week_sin, day_of_week_cos
  day_of_year -> day_of_year_sin, day_of_year_cos
  winddirection_10m_dominant -> winddirection_10m_dominant_sin, winddirection_10m_dominant_cos

All features: ['holiday', 'weathercode', 'temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'apparent_temperature_max', 'apparent_temperature_min', 'apparent_temperature_mean', 'sunrise', 'sunset', 'daylight_duration', 'sunshine_duration', 'rain_sum', 'snowfall_sum', 'precipitation_hours', 'windspeed_10m_max', 'windgusts_10m_max', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration', 'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos', 'winddirection_10m_dominant_sin', 'winddirection_10m_dominant_cos']


In [20]:
# Check feature ranges before scaling (continuous features only)
print("Continuous feature statistics before scaling:")
X_encoded[continuous_cols].describe().T[['min', 'max', 'mean', 'std']]


Continuous feature statistics before scaling:


,min,max,mean,std
temperature_2m_max,-6.70,36.10,14.928006,8.401043
temperature_2m_min,-17.50,23.10,7.235309,6.700360
temperature_2m_mean,-10.20,29.10,11.075225,7.463579
apparent_temperature_max,-13.00,36.70,12.627893,9.957706
apparent_temperature_min,-22.80,23.50,4.740225,8.176099
apparent_temperature_mean,-14.70,29.60,8.742556,9.013316
sunrise,9960.00,26400.00,17927.696629,5540.841244
sunset,53760.00,70560.00,62434.162921,5664.918279
daylight_duration,27607.04,60523.94,44506.771744,11157.694989
sunshine_duration,0.00,55333.02,27306.354107,17105.955470


In [21]:
# Apply StandardScaler ONLY to continuous features
scaler = StandardScaler()

# Start with the encoded data
X_scaled = X_encoded.copy()

# Scale only continuous columns
X_scaled[continuous_cols] = scaler.fit_transform(X_encoded[continuous_cols])

print("Feature preprocessing summary:")
print(f"  - Boolean features (unchanged): {boolean_cols}")
print(f"  - Categorical features (unchanged): {categorical_cols}")
print(f"  - Cyclical features (sin/cos encoded): {[f'{c}_sin, {c}_cos' for c in cyclical_cols]}")
print(f"  - Continuous features (standardized): {len(continuous_cols)} columns")
print(f"\nFinal feature matrix shape: {X_scaled.shape}")
print(f"\nContinuous feature statistics after scaling:")
X_scaled[continuous_cols].describe().T[['min', 'max', 'mean', 'std']]


Feature preprocessing summary:
  - Boolean features (unchanged): ['holiday']
  - Categorical features (unchanged): ['weathercode']
  - Cyclical features (sin/cos encoded): ['day_of_week_sin, day_of_week_cos', 'day_of_year_sin, day_of_year_cos', 'winddirection_10m_dominant_sin, winddirection_10m_dominant_cos']
  - Continuous features (standardized): 17 columns

Final feature matrix shape: (3560, 25)

Continuous feature statistics after scaling:


,min,max,mean,std
temperature_2m_max,-2.574805,2.520517,6.386901e-17,1.00014
temperature_2m_min,-3.692157,2.368070,1.596725e-17,1.00014
temperature_2m_mean,-2.850940,2.415370,-3.193450e-17,1.00014
apparent_temperature_max,-2.574036,2.417775,-6.386901e-17,1.00014
apparent_temperature_min,-3.368855,2.294788,6.786082e-17,1.00014
apparent_temperature_mean,-2.601246,2.314395,-3.193450e-17,1.00014
sunrise,-1.438196,1.529279,1.916070e-16,1.00014
sunset,-1.531422,1.434615,-5.029685e-16,1.00014
daylight_duration,-1.514838,1.435728,2.794269e-17,1.00014
sunshine_duration,-1.596531,1.638646,-1.437053e-16,1.00014


In [22]:
# Export preprocessed data to CSV
export_df = X_scaled.copy()
export_df['slp'] = y.values
export_df['date'] = df_clean['date'].values

# Reorder columns to put date first
cols = ['date', 'slp'] + [col for col in export_df.columns if col not in ['date', 'slp']]
export_df = export_df[cols]

# Save to dataset folder
export_df.to_csv('dataset/data_v2_step_5.csv', sep=';', decimal=',', index=False)
print(f"Exported preprocessed data to 'dataset/data_v2_step_5.csv'")
print(f"Shape: {export_df.shape}")


Exported preprocessed data to 'dataset/data_v2_step_5.csv'
Shape: (3560, 27)


## 3. Time Series Split

In [23]:
# Use TimeSeriesSplit for proper time series cross-validation
# This ensures we always train on past data and test on future data
tscv = TimeSeriesSplit(n_splits=10)

# Visualize the splits
print("Time Series Cross-Validation Splits:")
print("="*50)
for fold, (train_idx, test_idx) in enumerate(tscv.split(X_scaled)):
    train_dates = df_clean.iloc[train_idx]['date']
    test_dates = df_clean.iloc[test_idx]['date']
    print(f"Fold {fold + 1}:")
    print(f"  Train: {train_dates.min().date()} to {train_dates.max().date()} ({len(train_idx)} samples)")
    print(f"  Test:  {test_dates.min().date()} to {test_dates.max().date()} ({len(test_idx)} samples)")
    print()


Time Series Cross-Validation Splits:
Fold 1:
  Train: 2016-01-01 to 2016-11-25 (330 samples)
  Test:  2016-11-26 to 2017-10-14 (323 samples)

Fold 2:
  Train: 2016-01-01 to 2017-10-14 (653 samples)
  Test:  2017-10-15 to 2018-09-02 (323 samples)

Fold 3:
  Train: 2016-01-01 to 2018-09-02 (976 samples)
  Test:  2018-09-03 to 2019-07-22 (323 samples)

Fold 4:
  Train: 2016-01-01 to 2019-07-22 (1299 samples)
  Test:  2019-07-23 to 2020-06-09 (323 samples)

Fold 5:
  Train: 2016-01-01 to 2020-06-09 (1622 samples)
  Test:  2020-06-10 to 2021-04-28 (323 samples)

Fold 6:
  Train: 2016-01-01 to 2021-04-28 (1945 samples)
  Test:  2021-04-29 to 2022-03-17 (323 samples)

Fold 7:
  Train: 2016-01-01 to 2022-03-17 (2268 samples)
  Test:  2022-03-18 to 2023-02-03 (323 samples)

Fold 8:
  Train: 2016-01-01 to 2023-02-03 (2591 samples)
  Test:  2023-02-04 to 2023-12-23 (323 samples)

Fold 9:
  Train: 2016-01-01 to 2023-12-23 (2914 samples)
  Test:  2023-12-24 to 2024-11-10 (323 samples)

Fold 10:
  T

## 4. Define Models

In [24]:
# Define different types of models with standard parameters
models = {
    # Linear Models
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    
    # Tree-based Models
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42, n_jobs=-1, verbosity=0),
    'LightGBM': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
    
    # Distance-based Models
    'K-Nearest Neighbors': KNeighborsRegressor(),
    
    # Support Vector Machine (tuned for large target values)
    'SVR': SVR(C=1e6, epsilon=1e4, kernel='rbf'),
}

print(f"Total models to evaluate: {len(models)}")
for name in models.keys():
    print(f"  - {name}")


Total models to evaluate: 12
  - Linear Regression
  - Ridge Regression
  - Lasso Regression
  - ElasticNet
  - Decision Tree
  - Random Forest
  - Gradient Boosting
  - AdaBoost
  - XGBoost
  - LightGBM
  - K-Nearest Neighbors
  - SVR


## 5. Train and Test Models

In [25]:
def evaluate_model(model, X, y, tscv):
    """
    Evaluate a model using time series cross-validation.
    Returns average metrics across all folds.
    """
    rmse_scores = []
    mae_scores = []
    r2_scores = []
    
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        rmse_scores.append(rmse)
        mae_scores.append(mae)
        r2_scores.append(r2)
    
    return {
        'RMSE_mean': np.mean(rmse_scores),
        'RMSE_std': np.std(rmse_scores),
        'MAE_mean': np.mean(mae_scores),
        'MAE_std': np.std(mae_scores),
        'R2_mean': np.mean(r2_scores),
        'R2_std': np.std(r2_scores),
    }


In [26]:
# Train and evaluate all models
results = {}

print("Training and evaluating models...")
print("="*60)

print(X_scaled.head())
print(X_scaled.dtypes)
print(X_scaled.describe())

for name, model in models.items():
    print(f"Training: {name}...", end=" ")
    try:
        metrics = evaluate_model(model, X_scaled, y, tscv)
        results[name] = metrics
        print(f"Done! (R² = {metrics['R2_mean']:.4f})")
    except Exception as e:
        print(f"Error: {str(e)}")
        results[name] = {'RMSE_mean': np.nan, 'MAE_mean': np.nan, 'R2_mean': np.nan}

print("\nAll models trained!")


Training and evaluating models...
   holiday  weathercode  temperature_2m_max  temperature_2m_min  \
0        1            3           -1.265259           -1.065064   
1        0            3           -1.479549           -1.871104   
2        0            3           -2.420040           -2.333831   
3        0            3           -2.193846           -2.259197   
4        0           73           -2.396230           -2.035297   

   temperature_2m_mean  apparent_temperature_max  apparent_temperature_min  \
0            -1.162504                 -1.047366                 -0.971287   
1            -1.685115                 -1.338638                 -2.035514   
2            -2.395330                 -2.453509                 -2.390256   
3            -2.247927                 -2.182324                 -2.304629   
4            -2.261327                 -2.332983                 -2.059979   

   apparent_temperature_mean   sunrise    sunset  ...  windspeed_10m_max  \
0                 

## 6. Results


In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('R2_mean', ascending=False)

# Format for display
results_display = results_df.copy()
results_display['RMSE'] = results_display.apply(lambda x: f"{x['RMSE_mean']:.2f} ± {x['RMSE_std']:.2f}", axis=1)
results_display['MAE'] = results_display.apply(lambda x: f"{x['MAE_mean']:.2f} ± {x['MAE_std']:.2f}", axis=1)
results_display['R²'] = results_display.apply(lambda x: f"{x['R2_mean']:.4f} ± {x['R2_std']:.4f}", axis=1)

print("\n" + "="*80)
print("MODEL COMPARISON RESULTS (sorted by R² score)")
print("="*80)
print("\nMetrics averaged over 10-fold Time Series Cross-Validation:")
print()
results_display[['RMSE', 'MAE', 'R²']]


In [ ]:
# Summary statistics
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

best_model = results_df['R2_mean'].idxmax()
best_r2 = results_df.loc[best_model, 'R2_mean']
best_rmse = results_df.loc[best_model, 'RMSE_mean']
best_mae = results_df.loc[best_model, 'MAE_mean']

print(f"\nBest Model: {best_model}")
print(f"   - R² Score: {best_r2:.4f}")
print(f"   - RMSE: {best_rmse:.2f}")
print(f"   - MAE: {best_mae:.2f}")

print(f"\nTarget variable (slp) statistics:")
print(f"   - Mean: {y.mean():.2f}")
print(f"   - Std: {y.std():.2f}")
print(f"   - Min: {y.min():.2f}")
print(f"   - Max: {y.max():.2f}")


In [ ]:
# Visual comparison (text-based bar chart)
print("\n" + "="*80)
print("R² SCORE COMPARISON")
print("="*80 + "\n")

max_bar_length = 50
max_r2 = results_df['R2_mean'].max()

for model_name in results_df.index:
    r2 = results_df.loc[model_name, 'R2_mean']
    if r2 > 0:
        bar_length = int((r2 / max_r2) * max_bar_length)
        bar = '█' * bar_length
    else:
        bar_length = 0
        bar = ''
    print(f"{model_name:25s} | {bar} {r2:.4f}")


## 7. Optimal Training Timespan Analysis

This section determines the optimal amount of historical data for predicting one year ahead.
We use the last year of data as the test set and vary the training period from 1 year to all available historical data.


In [ ]:
# Define the test period: last 1 year of data
test_end_date = df_clean['date'].max()
test_start_date = test_end_date - pd.DateOffset(years=1)

# Create test set mask
test_mask = df_clean['date'] > test_start_date
X_test_final = X_scaled[test_mask]
y_test_final = y[test_mask]

print(f"Test period: {test_start_date.date()} to {test_end_date.date()}")
print(f"Test set size: {len(X_test_final)} samples")

# Available training data (everything before test period)
train_available_mask = df_clean['date'] <= test_start_date
train_start_date = df_clean[train_available_mask]['date'].min()
train_end_date = df_clean[train_available_mask]['date'].max()

print(f"\nAvailable training period: {train_start_date.date()} to {train_end_date.date()}")
total_train_years = (train_end_date - train_start_date).days / 365.25
print(f"Total available training data: {total_train_years:.1f} years ({train_available_mask.sum()} samples)")


In [ ]:
# Define the top 3 models based on previous results
top_models = {
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'LightGBM': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
    'Random Forest': RandomForestRegressor(random_state=42),
}

# Calculate max years available for training
max_years = int(total_train_years)
print(f"Testing training periods from 1 to {max_years} years\n")

# Store results for each training period
timespan_results = {model_name: {'years': [], 'rmse': [], 'mae': [], 'r2': []} 
                    for model_name in top_models.keys()}

print("Evaluating models with different training timespans...")
print("=" * 70)

for n_years in range(1, max_years + 1):
    # Define training period: n_years before test_start_date
    train_period_start = test_start_date - pd.DateOffset(years=n_years)
    
    # Create training mask for this period
    train_mask = (df_clean['date'] > train_period_start) & (df_clean['date'] <= test_start_date)
    
    X_train = X_scaled[train_mask]
    y_train = y[train_mask]
    
    print(f"\n{n_years} year(s) of training data: {train_period_start.date()} to {test_start_date.date()} ({len(X_train)} samples)")
    
    for model_name, model in top_models.items():
        # Clone the model to avoid refitting issues
        from sklearn.base import clone
        model_clone = clone(model)
        
        # Train and predict
        model_clone.fit(X_train, y_train)
        y_pred = model_clone.predict(X_test_final)
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test_final, y_pred))
        mae = mean_absolute_error(y_test_final, y_pred)
        r2 = r2_score(y_test_final, y_pred)
        
        # Store results
        timespan_results[model_name]['years'].append(n_years)
        timespan_results[model_name]['rmse'].append(rmse)
        timespan_results[model_name]['mae'].append(mae)
        timespan_results[model_name]['r2'].append(r2)
        
        print(f"  {model_name}: R² = {r2:.4f}, RMSE = {rmse:.2f}")

print("\n" + "=" * 70)
print("Training timespan analysis complete!")


In [ ]:
# Create a comprehensive results DataFrame
timespan_df_list = []
for model_name, results in timespan_results.items():
    for i in range(len(results['years'])):
        timespan_df_list.append({
            'Model': model_name,
            'Training Years': results['years'][i],
            'RMSE': results['rmse'][i],
            'MAE': results['mae'][i],
            'R²': results['r2'][i]
        })

timespan_df = pd.DataFrame(timespan_df_list)

# Pivot table for R² scores
r2_pivot = timespan_df.pivot(index='Training Years', columns='Model', values='R²')
rmse_pivot = timespan_df.pivot(index='Training Years', columns='Model', values='RMSE')

print("\n" + "=" * 80)
print("R² SCORES BY TRAINING TIMESPAN")
print("=" * 80)
print(r2_pivot.round(4).to_string())

print("\n" + "=" * 80)
print("RMSE BY TRAINING TIMESPAN")
print("=" * 80)
print(rmse_pivot.round(2).to_string())


In [ ]:
# Find optimal training timespan for each model
print("\n" + "=" * 80)
print("OPTIMAL TRAINING TIMESPAN ANALYSIS")
print("=" * 80)

optimal_timespans = {}
for model_name in top_models.keys():
    model_data = timespan_df[timespan_df['Model'] == model_name]
    best_idx = model_data['R²'].idxmax()
    best_row = timespan_df.loc[best_idx]
    optimal_timespans[model_name] = {
        'years': int(best_row['Training Years']),
        'r2': best_row['R²'],
        'rmse': best_row['RMSE'],
        'mae': best_row['MAE']
    }
    print(f"\n{model_name}:")
    print(f"  Optimal training period: {optimal_timespans[model_name]['years']} year(s)")
    print(f"  Best R²: {optimal_timespans[model_name]['r2']:.4f}")
    print(f"  RMSE: {optimal_timespans[model_name]['rmse']:.2f}")
    print(f"  MAE: {optimal_timespans[model_name]['mae']:.2f}")

# Calculate average R² for each training timespan across all models
avg_r2_by_years = timespan_df.groupby('Training Years')['R²'].mean()
optimal_years_overall = avg_r2_by_years.idxmax()
optimal_r2_overall = avg_r2_by_years.max()

print("\n" + "-" * 80)
print("OVERALL RECOMMENDATION (averaged across all 3 models)")
print("-" * 80)
print(f"\nOptimal training timespan: {optimal_years_overall} year(s)")
print(f"Average R² score: {optimal_r2_overall:.4f}")


In [ ]:
# Visual comparison of R² scores by training timespan
print("\n" + "=" * 80)
print("R² SCORE BY TRAINING TIMESPAN (Visual)")
print("=" * 80 + "\n")

for model_name in top_models.keys():
    print(f"\n{model_name}:")
    model_data = timespan_df[timespan_df['Model'] == model_name].sort_values('Training Years')
    
    max_r2 = model_data['R²'].max()
    for _, row in model_data.iterrows():
        years = int(row['Training Years'])
        r2 = row['R²']
        bar_length = int((r2 / max_r2) * 40) if r2 > 0 else 0
        bar = '█' * bar_length
        marker = ' ← BEST' if r2 == max_r2 else ''
        print(f"  {years:2d} year(s) | {bar} {r2:.4f}{marker}")

# Average across models
print(f"\nAverage (all models):")
for years in sorted(avg_r2_by_years.index):
    r2 = avg_r2_by_years[years]
    bar_length = int((r2 / avg_r2_by_years.max()) * 40) if r2 > 0 else 0
    bar = '█' * bar_length
    marker = ' ← BEST' if years == optimal_years_overall else ''
    print(f"  {years:2d} year(s) | {bar} {r2:.4f}{marker}")


In [ ]:
# Final summary
print("\n" + "=" * 80)
print("FINAL SUMMARY: OPTIMAL TRAINING TIMESPAN")
print("=" * 80)

print(f"""
Test Period: {test_start_date.date()} to {test_end_date.date()} ({len(y_test_final)} days)

Results by Model:
""")

for model_name, opt in optimal_timespans.items():
    print(f"  • {model_name}: {opt['years']} year(s) → R² = {opt['r2']:.4f}")

print(f"""
════════════════════════════════════════════════════════════════════════════════
RECOMMENDATION: Use {optimal_years_overall} year(s) of historical data for training
                when predicting 1 year ahead.
                
                This achieves an average R² of {optimal_r2_overall:.4f} across the
                top 3 models (Random Forest, Gradient Boosting, LightGBM).
════════════════════════════════════════════════════════════════════════════════
""")


In [ ]:
# Export data with optimal training timespan + test timespan
# Define the optimal training period start date
optimal_train_start = test_start_date - pd.DateOffset(years=optimal_years_overall)

# Create mask for optimal training + test data
export_mask = df_clean['date'] > optimal_train_start

# Create export dataframe with scaled features
export_df_optimal = X_scaled[export_mask].copy()
export_df_optimal['slp'] = y[export_mask].values
export_df_optimal['date'] = df_clean.loc[export_mask, 'date'].values

# Reorder columns to put date first
cols = ['date', 'slp'] + [col for col in export_df_optimal.columns if col not in ['date', 'slp']]
export_df_optimal = export_df_optimal[cols]

# Save to CSV
export_path = 'dataset/data_v3_full.csv'
export_df_optimal.to_csv(export_path, sep=';', decimal=',', index=False)

print(f"Exported data to '{export_path}'")
print(f"\nData period: {optimal_train_start.date()} to {test_end_date.date()}")
print(f"  - Training period: {optimal_train_start.date()} to {test_start_date.date()} ({optimal_years_overall} years)")
print(f"  - Test period: {test_start_date.date()} to {test_end_date.date()} (1 year)")
print(f"\nTotal samples: {len(export_df_optimal)}")
print(f"Shape: {export_df_optimal.shape}")